## FUNCION 

Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta pero con alta cardinalidad y una variable float "pvalue" cuyo valor por defecto será 0.05.

La función debe devolver una lista con las columnas categóricas del dataframe cuyo test de relación con la columna designada por 'target_col' supere en confianza estadística el test de relación que sea necesario hacer (es decir la función debe poder escoger cuál de los dos test que hemos aprendido tiene que hacer).

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.

#### pasos a seguir
Argumentos: 
    df, target_col, pvalue = 00.5
devolver:
    lista de columnas categóricas cuyo test de relación con la columna targer supera la confianza estadistica del teste de relafcion 


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency



### Funcion: get_features_cat_classification

Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de clasificación, es decir debe ser una variable categórica o numérica discreta pero con baja cardinalidad, un argumento "normalize" con valor False por defecto, una variable float "mi_threshold" cuyo valor por defecto será 0.

* En caso de que "normalize" sea False:
    La función debe devolver una lista con las columnas categóricas del dataframe cuyo valor de mutual information con 'target_col' iguale o supere el valor de "mi_threshold".

* En caso de que "normalize" sea True:
    La función debe devolver una lista con las columnas categóricas del dataframe cuyo valor normalizado de mutual information con 'target_col' iguale o supere el valor de "mi_threshold". El valor normalizado de mutual information se considera el obtenido de dividir el valor de mutual information tal cual ofrece sklearn o la fórmula de cálculo por la suma de todos los valores de mutual information de las features categóricas del dataframe.
    En este caso, la función debe comprobar que "mi_threshold" es un valor float entre 0 y 1, y arrojar un error si no lo es.

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable categórica del dataframe.

In [1]:
from sklearn.feature_selection import mutual_info_classif
import numpy as np

def get_features_cat_classification(df, target_col, normalize=False, mi_threshold=0):
    # Comprobar si 'target_col' es una columna del DataFrame
    if target_col not in df.columns:
        print(f"Error: '{target_col}' no es una columna válida en el DataFrame.")
        return None

    # Comprobar si 'target_col' es una variable categórica
    if not pd.api.types.is_categorical_dtype(df[target_col]) and not pd.api.types.is_object_dtype(df[target_col]):
        print(f"Error: '{target_col}' no es una variable categórica en el DataFrame.")
        return None

    # Comprobar si 'mi_threshold' es un valor float entre 0 y 1
    if not isinstance(mi_threshold, float) or mi_threshold < 0 or mi_threshold > 1:
        print("Error: 'mi_threshold' debe ser un valor float entre 0 y 1.")
        return None

    # Comprobar si 'normalize' es un booleano
    if not isinstance(normalize, bool):
        print("Error: 'normalize' debe ser un valor booleano (True/False).")
        return None

    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    X_cat = df[categorical_cols]
    y = df[target_col]

    # Calcula la información mutua
    mi_scores = mutual_info_classif(X_cat, y)

    if normalize:
        # Calcula la suma de los valores de información mutua
        mi_sum = np.sum(mi_scores)

        # Normaliza los valores de información mutua
        mi_scores = mi_scores / mi_sum

    # Filtra las características que superan el umbral
    selected_features = [categorical_cols[i] for i, mi_score in enumerate(mi_scores) if mi_score >= mi_threshold]

    return selected_features

# Ejemplo de uso
# selected_features = get_features_cat_classification(df, 'target_column', normalize=True, mi_threshold=0.1)
# print(selected_features)


### Funcion: plot_features_cat_classification

Esta función recibe un dataframe, una argumento "target_col" con valor por defecto "", una lista de strings ("columns") cuyo valor por defecto es la lista vacía, un argumento ("mi_threshold") con valor 0.0 por defecto, y un argumento "normalize" a False.

Si la lista no está vacía:
* La función seleccionara de esta lista los valores que correspondan a columnas o features categóricas del dataframe cuyo valor de mutual information respecto de target_col supere el umbral puesto en "mi_threshold" (con las mismas considereciones respecto a "normalize" que se comentan en la descripción de la función "get_features_cat_classification").
* Para los valores seleccionados, pintará la distribución de etiquetas de cada valor respecto a los valores de la columna "target_col".

Si la lista está vacía:
* Entonces la función igualará "columns" a las variables categóricas del dataframe y se comportará como se describe en la sección "Si la lista no está vacía"

De igual manera que en la función descrita anteriormente deberá hacer un check de los valores de entrada y comportarse como se describe en el último párrafo de la función `get_features_cat_classification`.

In [2]:
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def visualize_cat_features(df, target_col="", columns=[], mi_threshold=0.0, normalize=False):
    # Comprobar si 'target_col' es una columna del DataFrame
    if target_col not in df.columns:
        print(f"Error: '{target_col}' no es una columna válida en el DataFrame.")
        return None

    # Comprobar si 'target_col' es una variable categórica
    if not pd.api.types.is_categorical_dtype(df[target_col]) and not pd.api.types.is_object_dtype(df[target_col]):
        print(f"Error: '{target_col}' no es una variable categórica en el DataFrame.")
        return None

    # Comprobar si 'mi_threshold' es un valor float entre 0 y 1 si normalize es True
    if normalize and (not isinstance(mi_threshold, float) or mi_threshold < 0 or mi_threshold > 1):
        print("Error: 'mi_threshold' debe ser un valor float entre 0 y 1 si normalize es True.")
        return None

    # Si la lista de columnas está vacía, seleccionar todas las variables categóricas del dataframe
    if not columns:
        columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

    selected_features = []

    for col in columns:
        # Comprobar si la columna está en el dataframe
        if col not in df.columns:
            print(f"La columna '{col}' no está presente en el DataFrame. Se omitirá.")
            continue

        # Calcular la información mutua para la columna actual
        mi_score = mutual_info_classif(df[[col]], df[target_col])[0]

        if mi_score >= mi_threshold:
            selected_features.append(col)

            # Visualizar la distribución de etiquetas respecto a target_col
            plt.figure(figsize=(8, 6))
            sns.countplot(x=col, hue=target_col, data=df)
            plt.title(f"Distribución de '{col}' respecto a '{target_col}'")
            plt.show()

    return selected_features

# Ejemplo de uso
# selected_features = visualize_cat_features(df, target_col="target_column", columns=["feature1", "feature2"], mi_threshold=0.1, normalize=True)
# print(selected_features)


In [5]:
df = pd.read_csv("./data3/titanic.csv")
target_col = "alive"

In [7]:
get_features_cat_classification(df, target_col, normalize=False, mi_threshold=0.1)

C:\Users\adrif\AppData\Local\Temp\ipykernel_32076\190829794.py:11: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[target_col]) and not pd.api.types.is_object_dtype(df[target_col]):


ValueError: could not convert string to float: 'male'

In [8]:
visualize_cat_features(df, target_col, columns=[], mi_threshold=0.0, normalize=False)

C:\Users\adrif\AppData\Local\Temp\ipykernel_32076\3237113076.py:14: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[target_col]) and not pd.api.types.is_object_dtype(df[target_col]):


ValueError: could not convert string to float: 'male'